In [ ]:
import pandas as pd
import tqdm
import ast
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score
import re
import numpy as np

In [5]:
path = 'new_books_prepared.csv'

df = pd.read_csv(path, index_col=0)
df['tokens'] = df.tokens.apply(ast.literal_eval)
df['labels'] = df.labels.apply(ast.literal_eval)
df = df[df.tokens.apply(len) < 200]
df_train, df_val_test = train_test_split(df, test_size=0.2, random_state=999)
df_val, df_test  = train_test_split(df_val_test, test_size=0.5, random_state=999)

df_test

,text,tokens,clear_punct_lower,labels
1341,Все это смог я различить лишь смутно и с трудо...,"[все, это, смог, я, различить, лишь, смутно, и...",все это смог я различить лишь смутно и с трудо...,"[o, o, o, o, o, o, o, o, o, ., o, o, o, o, o, ..."
25027,"Она поехала в игрушечную лавку, накупила игруш...","[она, поехала, в, игрушечную, лавку, накупила,...",она поехала в игрушечную лавку накупила игруше...,"[o, o, o, o, ,, o, o, o, o, o, ., o, o, o, ,, ..."
2585,Наконец настало утро четырнадцатого числа. пог...,"[наконец, настало, утро, четырнадцатого, числа...",наконец настало утро четырнадцатого числа пого...,"[o, o, o, o, ., o, o, o, o, o, o, o, ,, o, o, ..."
16829,"Хорошо. А почему прежде, бывало, с восьми часо...","[хорошо, а, почему, прежде, бывало, с, восьми,...",хорошо а почему прежде бывало с восьми часов в...,"[., o, o, ,, ,, o, o, o, o, o, o, o, ,, o, o, ..."
7937,"Говоря это, графиня оглянулась на дочь. Наташа...","[говоря, это, графиня, оглянулась, на, дочь, н...",говоря это графиня оглянулась на дочь наташа л...,"[o, ,, o, o, o, ., o, ,, o, o, o, o, o, o, o, ..."
...,...,...,...,...
13908,Разве на одну секунду... Я пришел за советом. ...,"[разве, на, одну, секунду, я, пришел, за, сове...",разве на одну секунду я пришел за советом я ко...,"[o, o, o, ..., o, o, o, ., ,, ,, o, o, o, ,, ,..."
21490,"План был очень хорош, но дело заключалось в то...","[план, был, очень, хорош, но, дело, заключалос...",план был очень хорош но дело заключалось в том...,"[o, o, o, ,, o, o, o, o, ,, o, o, o, o, o, o, ..."
2567,"Сохраняя, поелику возможно, равновесие, чтобы ...","[сохраняя, поелику, возможно, равновесие, чтоб...",сохраняя поелику возможно равновесие чтобы хор...,"[,, o, ,, ,, o, o, o, ,, o, o, ,, o, o, o, o, ..."
25405,"Было ли в лице Левина что-нибудь особенное, ил...","[было, ли, в, лице, левина, чтонибудь, особенн...",было ли в лице левина чтонибудь особенное или ...,"[o, o, o, o, o, o, ,, o, o, o, ,, o, o, o, o, ..."


In [7]:
def calc_metrics(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    print('Доля пробелов:', (y_true == 7).mean())
    metrics = []
    enctryption = {0:",",1:':',2:';',3:".",4:"!",5:"?",6:'...', 7:'o'}
    sorted_dict = {enctryption[i[0]] : i[1] for i in sorted(Counter(y_true).items())}
    
    metrics.append(list(dict(sorted(Counter(y_true).items())).values()))
    metrics.append(f1_score(y_true, y_pred, average=None))
    metrics.append(precision_score(y_true, y_pred, average=None, zero_division=0))
    metrics.append(recall_score(y_true, y_pred, average=None, zero_division=0))
    metrics_index = ['Count', 'F1-Score', 'Precision', 'Recall']
    df_metrics = pd.DataFrame(metrics, columns=sorted_dict.keys(), index=metrics_index)
    
    return df_metrics

## Xlm-roberta_punctuation


In [11]:
from punctuators.models import PunctCapSegModelONNX

m = PunctCapSegModelONNX.from_pretrained(
    "1-800-BAD-CODE/xlm-roberta_punctuation_fullstop_truecase"
)

input_texts = [
    'привет как дела это новый кадиллак'
]

results = m.infer(
    texts=input_texts, apply_sbd=True,
)

' '.join(results[0])

sp.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/531 [00:00<?, ?B/s]

/home/jovyan/.mlspace/envs/kozlov-ky/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:121: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


'Привет, как дела? Это новый кадиллак.'

In [12]:
punctuation_signs = ['!', ',', '.', '...', ':', ';', '?']

def roberta_prediction(text):
    text = re.sub('– ', '', text)
    text = re.sub('— ', '', text)
    text = re.sub('"', '', text)
    text = text.lower()
    text = re.sub('\s+', ' ', text)

    for sign in punctuation_signs:
        text = text.replace(sign + ' ', ' ')
        
    if text[-1] in punctuation_signs:
        text = text[:-1]
        
    preds = m.infer(
    texts=[text], apply_sbd=False,
    )
    prediction = preds[0]
    tokens = [token for token in prediction.split(' ') if token != '']
    labels = []
    
    for token in tokens:
        if (len(token) > 3) & (token[-3:] == '...'):
            labels.append('...')
        elif token[-1] in punctuation_signs:
            labels.append(token[-1])
        else:
            labels.append('o')
            
    
    return labels

In [13]:
preds = []
true_labels = []

for id_text in tqdm.tqdm(range(len(df_test.text.values))):
    if len(df_test.text.values[id_text]) < 10:
        continue
        
    prediction= roberta_prediction(df_test.text.values[id_text])
    needed_labels = df_test.labels.values[id_text]
    
    if len(prediction) != len(needed_labels):        
        not_empty_token_idxs = ~(np.array(df_test.tokens.values[id_text]) == '')
        needed_labels = np.array(needed_labels)[not_empty_token_idxs].tolist()
        
    if len(needed_labels) == len(prediction):
        true_labels += needed_labels
        preds += prediction

100%|██████████| 2586/2586 [20:51<00:00,  2.07it/s]


In [14]:
le = LabelEncoder().fit(true_labels)

In [15]:
y_pred = le.transform(preds)
y_true = le.transform(true_labels)

In [16]:
def calc_metrics_no_proba(y_true, y_pred):
    print('Доля пробелов:', (y_true == 6).mean())
    
    metrics = []
    metrics.append(list(dict(sorted(Counter(y_true).items())).values()))
    metrics.append(f1_score(y_true, y_pred, average=None))
    metrics.append(precision_score(y_true, y_pred, average=None, zero_division=0))
    metrics.append(recall_score(y_true, y_pred, average=None, zero_division=0))
    metrics_index = ['Count', 'F1-Score', 'Precision', 'Recall']
    df_metrics = pd.DataFrame(metrics, columns=le.classes_, index=metrics_index)
    
    return df_metrics

In [17]:
calc_metrics_no_proba(y_true, y_pred)

Доля пробелов: 0.7975428559669088


,!,",",.,...,:,?,o
Count,205.0,13002.000000,5794.000000,164.000000,214.0,297.000000,77510.000000
F1-Score,0.0,0.812895,0.727008,0.783883,0.0,0.616372,0.976453
Precision,0.0,0.791148,0.720780,0.981651,0.0,0.588957,0.978929
Recall,0.0,0.835871,0.733345,0.652439,0.0,0.646465,0.973990
